# 实践实验：决策树

在本练习中，您将从零开始实现决策树，并将其应用于分类蘑菇是否可食用的任务。

# 大纲
- [ 1 - 包 ](#1)
- [ 2 - 问题陈述](#2)
- [ 3 - 数据集](#3)
  - [ 3.1 独热编码数据集](#3.1)
- [ 4 - 决策树回顾](#4)
  - [ 4.1 计算熵](#4.1)
    - [ 练习 1](#ex01)
  - [ 4.2 分割数据集](#4.2)
    - [ 练习 2](#ex02)
  - [ 4.3 计算信息增益](#4.3)
    - [ 练习 3](#ex03)
  - [ 4.4 获取最佳分割](#4.4)
    - [ 练习 4](#ex04)
- [ 5 - 构建树](#5)


<a name="1"></a>
## 1 - 包 

首先，让我们运行下面的单元格，导入本次作业中需要的所有包。
- [numpy](https://www.numpy.org) 是 Python 中处理矩阵的基础包。
- [matplotlib](https://matplotlib.org) 是 Python 中用于绘制图形的著名库。
- ``utils.py`` 包含本次作业的辅助函数。您不需要修改此文件中的代码。


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *

%matplotlib inline

<a name="2"></a>
## 2 - 问题陈述

假设您正在创办一家种植和销售野生蘑菇的公司。 
- 由于并非所有蘑菇都可食用，您希望能够根据给定蘑菇的物理属性来判断它是否可食用或有毒
- 您有一些可用于此任务的现有数据。 

您可以使用数据来帮助识别哪些蘑菇可以安全销售吗？ 

注意：使用的数据集仅用于说明目的。它不旨在作为识别可食用蘑菇的指南。



<a name="3"></a>
## 3 - 数据集

您将首先加载此任务的数据集。您收集的数据集如下：

| 菌盖颜色 | 菌柄形状 | 单独生长 | 可食用 |
|:---------:|:-----------:|:--------:|:------:|
|   棕色   |   渐细  |    是   |    1   |
|   棕色   |  扩大  |    是   |    1   |
|   棕色   |  扩大  |    否    |    0   |
|   棕色   |  扩大  |    否    |    0   |
|   棕色   |   渐细  |    是   |    1   |
|    红色    |   渐细  |    是   |    0   |
|    红色    |  扩大  |    否    |    0   |
|   棕色   |  扩大  |    是   |    1   |
|    红色    |   渐细  |    否    |    1   |
|   棕色   |  扩大  |    否    |    0   |


-  您有 10 个蘑菇样本。对于每个样本，您有
    - 三个特征
        - 菌盖颜色（`棕色` 或 `红色`），
        - 菌柄形状（`渐细` 或 `扩大`），以及
        - 单独生长（`是` 或 `否`）
    - 标签
        - 可食用（`1` 表示是或 `0` 表示有毒）

<a name="3.1"></a>
### 3.1 独热编码数据集
为了便于实现，我们对特征进行了独热编码（将它们转换为 0 或 1 值的特征）

| 棕色菌盖 | 渐细菌柄形状 | 单独生长 | 可食用 |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |

因此，
- `X_train` 包含每个样本的三个特征 
    - 棕色（值为 `1` 表示"棕色"菌盖颜色，`0` 表示"红色"菌盖颜色）
    - 渐细形状（值为 `1` 表示"渐细菌柄形状"，`0` 表示"扩大"菌柄形状）
    - 单独生长（值为 `1` 表示"是"，`0` 表示"否"）

- `y_train` 是蘑菇是否可食用 
    - `y = 1` 表示可食用
    - `y = 0` 表示有毒

In [7]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

#### 查看变量
让我们更熟悉您的数据集。  
- 一个好的起点是打印出每个变量并查看它包含的内容。

下面的代码打印 `X_train` 的前几个元素和变量的类型。

In [8]:
print("First few elements of X_train:\n", X_train[:5])
print("Type of X_train:",type(X_train))

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Type of X_train: <class 'numpy.ndarray'>


现在，让我们对 `y_train` 做同样的事情

In [9]:
print("First few elements of y_train:", y_train[:5])
print("Type of y_train:",type(y_train))

First few elements of y_train: [1 1 0 0 1]
Type of y_train: <class 'numpy.ndarray'>


#### 检查变量的维度

另一种熟悉数据的有用方法是查看其维度。

请打印 `X_train` 和 `y_train` 的形状，并查看数据集中有多少个训练样本。

In [10]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


<a name="4"></a>
## 4 - 决策树回顾

在本实践实验中，您将基于提供的数据集构建决策树。

- 回想一下，构建决策树的步骤如下：
    - 从根节点的所有样本开始
    - 计算在所有可能特征上分割的信息增益，并选择信息增益最高的特征
    - 根据所选特征分割数据集，并创建树的左右分支
    - 继续重复分割过程，直到满足停止条件
  
  
- 在本实验中，您将实现以下函数，这些函数将让您使用信息增益最高的特征将节点分割为左右分支
    - 计算节点的熵 
    - 根据给定特征将节点处的数据集分割为左右分支
    - 计算在给定特征上分割的信息增益
    - 选择使信息增益最大化的特征
    
- 然后，我们将使用您实现的辅助函数，通过重复分割过程来构建决策树，直到满足停止条件 
    - 对于本实验，我们选择的停止条件是设置最大深度为 2

<a name="4.1"></a>
### 4.1 计算熵

首先，您将编写一个名为 `compute_entropy` 的辅助函数，该函数计算节点的熵（不纯度的度量）。 
- 该函数接受一个 numpy 数组（`y`），该数组指示该节点中的样本是否可食用（`1`）或有毒（`0`） 

完成下面的 `compute_entropy()` 函数以：
* 计算 $p_1$，即可食用样本的比例（即在 `y` 中值为 `1` 的样本）
* 然后熵计算为 

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$
* 注意 
    * 对数以 $2$ 为底计算
    * 为了实现目的，$0\text{log}_2(0) = 0$。也就是说，如果 `p_1 = 0` 或 `p_1 = 1`，将熵设置为 `0`
    * 确保检查节点处的数据不为空（即 `len(y) != 0`）。如果为空，返回 `0`
    
<a name="ex01"></a>
### 练习 1

请使用前面的说明完成 `compute_entropy()` 函数。
    
如果您遇到困难，可以查看下面单元格后提供的提示以帮助您实现。

In [22]:
# UNQ_C1
# GRADED FUNCTION: compute_entropy

def compute_entropy(y):
    """
    Computes the entropy for 
    
    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)
       
    Returns:
        entropy (float): Entropy at that node
        
    """
    # You need to return the following variables correctly
    entropy = 0.
    
    ### START CODE HERE ###
    if len(y) != 0:
        p1 = p1 = len(y[y == 1]) / len(y) 
     # For p1 = 0 and 1, set the entropy to 0 (to handle 0log0)
        if p1 != 0 and p1 != 1:
             entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
        else:
             entropy = 0
    ### END CODE HERE ###        
    
    return entropy

<details>
  <summary><font size="3" color="darkgreen"><b>点击查看提示</b></font></summary>
    
    
   * 要计算 `p1`
       * 您可以将 `y` 中值为 `1` 的样本子集获取为 `y[y == 1]`
       * 您可以使用 `len(y)` 来获取 `y` 中的样本数量
   * 要计算 `entropy`
       * <a href="https://numpy.org/doc/stable/reference/generated/numpy.log2.html">np.log2</a> 允许您计算 numpy 数组的以 2 为底的对数
       * 如果 `p1` 的值为 0 或 1，请确保将熵设置为 `0` 
     
    <details>
          <summary><font size="2" color="darkblue"><b> 点击查看更多提示</b></font></summary>
        
    * 以下是您如何构建此函数的整体实现
    ```python 
    def compute_entropy(y):
        
        # 您需要正确返回以下变量
        entropy = 0.

        ### START CODE HERE ###
        if len(y) != 0:
            # 您的代码：计算可食用样本的比例（即在 y 中值为 1 的样本）
            p1 =

            # 对于 p1 = 0 和 1，将熵设置为 0（以处理 0log0）
            if p1 != 0 and p1 != 1:
                # 您的代码：使用上面提供的公式计算熵
                entropy = 
            else:
                entropy = 0. 
        ### END CODE HERE ###        

        return entropy
    ```
    
    如果您仍然遇到困难，可以查看下面提供的提示，以了解如何计算 `p1` 和 `entropy`。
    
    <details>
          <summary><font size="2" color="darkblue"><b>计算 p1 的提示</b></font></summary>
           &emsp; &emsp; 您可以将 p1 计算为 <code>p1 = len(y[y == 1]) / len(y) </code>
    </details>

     <details>
          <summary><font size="2" color="darkblue"><b>计算 entropy 的提示</b></font></summary>
          &emsp; &emsp; 您可以将 entropy 计算为 <code>entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)</code>
    </details>
        
    </details>

</details>

    


您可以通过运行以下测试代码来检查您的实现是否正确：

In [23]:
# Compute entropy at the root node (i.e. with all examples)
# Since we have 5 edible and 5 non-edible mushrooms, the entropy should be 1"

print("Entropy at root node: ", compute_entropy(y_train)) 

# UNIT TESTS
compute_entropy_test(compute_entropy)

Entropy at root node:  1.0
 All tests passed.


**预期输出**：
<table>
  <tr>
    <td> <b>根节点处的熵：<b> 1.0 </td> 
  </tr>
</table>

<a name="4.2"></a>
### 4.2 分割数据集

接下来，您将编写一个名为 `split_dataset` 的辅助函数，该函数接受节点处的数据和要分割的特征，并将其分割为左右分支。稍后在实验中，您将实现代码来计算分割的好坏。

- 该函数接受训练数据、该节点处数据点的索引列表以及要分割的特征。 
- 它分割数据并返回左右分支处的索引子集。
- 例如，假设我们从根节点开始（所以 `node_indices = [0,1,2,3,4,5,6,7,8,9]`），我们选择在特征 `0` 上分割，即样本是否有棕色菌盖。
    - 然后函数的输出是 `left_indices = [0,1,2,3,4,7,9]` 和 `right_indices = [5,6,8]`
    
| 索引 | 棕色菌盖 | 渐细菌柄形状 | 单独生长 | 可食用 |
|:-----:|:---------:|:--------------------:|:--------:|:------:|
|   0   |     1     |           1          |     1    |    1   |
|   1   |     1     |           0          |     1    |    1   |
|   2   |     1     |           0          |     0    |    0   |
|   3   |     1     |           0          |     0    |    0   |
|   4   |     1     |           1          |     1    |    1   |
|   5   |     0     |           1          |     1    |    0   |
|   6   |     0     |           0          |     0    |    0   |
|   7   |     1     |           0          |     1    |    1   |
|   8   |     0     |           1          |     0    |    1   |
|   9   |     1     |           0          |     0    |    0   |

<a name="ex02"></a>
### 练习 2

请完成下面显示的 `split_dataset()` 函数

- 对于 `node_indices` 中的每个索引
    - 如果 `X` 在该索引处该特征的值是 `1`，将索引添加到 `left_indices`
    - 如果 `X` 在该索引处该特征的值是 `0`，将索引添加到 `right_indices`

如果您遇到困难，可以查看下面单元格后提供的提示以帮助您实现。

In [24]:
# UNQ_C2
# GRADED FUNCTION: split_dataset

def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):  List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (list): Indices with feature value == 1
        right_indices (list): Indices with feature value == 0
    """
    
    # You need to return the following variables correctly
    left_indices = []
    right_indices = []
    
    ### START CODE HERE ###
    for i in node_indices:   
        if X[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    ### END CODE HERE ###
        
    return left_indices, right_indices

<details>
  <summary><font size="3" color="darkgreen"><b>点击查看提示</b></font></summary>
    
    
   * 以下是您如何构建此函数的整体实现
    ```python 
    def split_dataset(X, node_indices, feature):
    
        # 您需要正确返回以下变量
        left_indices = []
        right_indices = []

        ### START CODE HERE ###
        # 遍历该节点处的样本索引
        for i in node_indices:   
            if # 您的代码：检查 X 在该索引处该特征的值是否为 1
                left_indices.append(i)
            else:
                right_indices.append(i)
        ### END CODE HERE ###
        
    return left_indices, right_indices
    ```
    <details>
          <summary><font size="2" color="darkblue"><b> 点击查看更多提示</b></font></summary>
        
    条件是 <code> if X[i][feature] == 1:</code>。
        
    </details>

</details>

    


现在，让我们使用下面的代码块检查您的实现。让我们尝试在根节点处分割数据集，该节点包含特征 0（棕色菌盖）的所有样本，正如我们上面讨论的那样

In [25]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Feel free to play around with these variables
# The dataset only has three features, so this value can be 0 (Brown Cap), 1 (Tapering Stalk Shape) or 2 (Solitary)
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# UNIT TESTS    
split_dataset_test(split_dataset)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
 All tests passed.


**预期输出**：
```
Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
```

<a name="4.3"></a>
### 4.3 计算信息增益

接下来，您将编写一个名为 `information_gain` 的函数，该函数接受训练数据、节点处的索引和要分割的特征，并返回分割的信息增益。

<a name="ex03"></a>
### 练习 3

请完成下面显示的 `compute_information_gain()` 函数以计算

$$\text{信息增益} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

其中 
- $H(p_1^\text{node})$ 是节点处的熵 
- $H(p_1^\text{left})$ 和 $H(p_1^\text{right})$ 是分割产生的左右分支处的熵
- $w^{\text{left}}$ 和 $w^{\text{right}}$ 分别是左右分支处的样本比例

注意：
- 您可以使用上面实现的 `compute_entropy()` 函数来计算熵
- 我们提供了一些起始代码，使用您上面实现的 `split_dataset()` 函数来分割数据集 

如果您遇到困难，可以查看下面单元格后提供的提示以帮助您实现。

In [26]:
# UNQ_C3
# GRADED FUNCTION: compute_information_gain

def compute_information_gain(X, y, node_indices, feature):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # You need to return the following variables correctly
    information_gain = 0
    
    ### START CODE HERE ###
    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)
    
    # Weights 
    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)
    
    #Weighted entropy
    weighted_entropy = w_left * left_entropy + w_right * right_entropy
    
    #Information gain 
    information_gain = node_entropy - weighted_entropy
    
    ### END CODE HERE ###  
    
    return information_gain

<details>
  <summary><font size="3" color="darkgreen"><b>点击查看提示</b></font></summary>
    
    
   * 以下是您如何构建此函数的整体实现
    ```python 
    def compute_information_gain(X, y, node_indices, feature):
        # 分割数据集
        left_indices, right_indices = split_dataset(X, node_indices, feature)

        # 一些有用的变量
        X_node, y_node = X[node_indices], y[node_indices]
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        # 您需要正确返回以下变量
        information_gain = 0

        ### START CODE HERE ###
        # 您的代码：使用 compute_entropy() 计算节点处的熵
        node_entropy = 
        # 您的代码：计算左分支处的熵
        left_entropy = 
        # 您的代码：计算右分支处的熵
        right_entropy = 

        # 您的代码：计算左分支处的样本比例
        w_left = 
        
        # 您的代码：计算右分支处的样本比例
        w_right = 

        # 您的代码：使用 w_left、w_right、left_entropy 和 right_entropy 计算分割的加权熵
        weighted_entropy = 

        # 您的代码：将信息增益计算为节点处的熵
        # 减去加权熵
        information_gain = 
        ### END CODE HERE ###  

        return information_gain
    ```
    如果您仍然遇到困难，请查看下面的提示。
    
    <details>
          <summary><font size="2" color="darkblue"><b> 计算熵的提示</b></font></summary>
        
    <code>node_entropy = compute_entropy(y_node)</code><br>
    <code>left_entropy = compute_entropy(y_left)</code><br>
    <code>right_entropy = compute_entropy(y_right)</code>
        
    </details>
    
    <details>
          <summary><font size="2" color="darkblue"><b>计算 w_left 和 w_right 的提示</b></font></summary>
           <code>w_left = len(X_left) / len(X_node)</code><br>
           <code>w_right = len(X_right) / len(X_node)</code>
    </details>
    
    <details>
          <summary><font size="2" color="darkblue"><b>计算 weighted_entropy 的提示</b></font></summary>
           <code>weighted_entropy = w_left * left_entropy + w_right * right_entropy</code>
    </details>
    
    <details>
          <summary><font size="2" color="darkblue"><b>计算 information_gain 的提示</b></font></summary>
           <code> information_gain = node_entropy - weighted_entropy</code>
    </details>


</details>


您现在可以使用下面的单元格检查您的实现，并计算在每个特征上分割的信息增益

In [27]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)
    
info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

# UNIT TESTS
compute_information_gain_test(compute_information_gain)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377
 All tests passed.


**预期输出**：
```
Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377
```

在根节点处分割"单独生长"（特征 = 2）给出最大信息增益。因此，它是在根节点处分割的最佳特征。

<a name="4.4"></a>
### 4.4 获取最佳分割
现在让我们编写一个函数，通过计算每个特征的信息增益（如上面所做的那样）来获取最佳分割特征，并返回给出最大信息增益的特征

<a name="ex04"></a>
### 练习 4
请完成下面显示的 `get_best_split()` 函数。
- 该函数接受训练数据以及该节点处数据点的索引
- 函数的输出是给出最大信息增益的特征 
    - 您可以使用 `compute_information_gain()` 函数遍历特征并计算每个特征的信息增益
如果您遇到困难，可以查看下面单元格后提供的提示以帮助您实现。

In [36]:
# UNQ_C4
# GRADED FUNCTION: get_best_split

def get_best_split(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    
    ### START CODE HERE ###
    max_info_gain=0
    for feature in range(num_features):
        info_gain = compute_information_gain(X, y, node_indices, feature)
        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_feature = feature
                
        
    ### END CODE HERE ##    
       
    return best_feature


<details>
  <summary><font size="3" color="darkgreen"><b>点击查看提示</b></font></summary>
    
    
   * 以下是您如何构建此函数的整体实现
    
    ```python 
    def get_best_split(X, y, node_indices):   

        # 一些有用的变量
        num_features = X.shape[1]

        # 您需要正确返回以下变量
        best_feature = -1

        ### START CODE HERE ###
        max_info_gain = 0

        # 遍历所有特征
        for feature in range(num_features): 
            
            # 您的代码：计算在此特征上分割的信息增益
            info_gain = 
            
            # 如果信息增益大于目前看到的最大值
            if info_gain > max_info_gain:  
                # 您的代码：设置 max_info_gain 和 best_feature
                max_info_gain = 
                best_feature = 
        ### END CODE HERE ##    
   
    return best_feature
    ```
    如果您仍然遇到困难，请查看下面的提示。
    
    <details>
          <summary><font size="2" color="darkblue"><b> 计算 info_gain 的提示</b></font></summary>
        
    <code>info_gain = compute_information_gain(X, y, node_indices, feature)</code>
    </details>
    
    <details>
          <summary><font size="2" color="darkblue"><b>更新 max_info_gain 和 best_feature 的提示</b></font></summary>
           <code>max_info_gain = info_gain</code><br>
           <code>best_feature = feature</code>
    </details>
</details>


现在，让我们使用下面的单元格检查您的函数实现。

In [37]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

# UNIT TESTS
get_best_split_test(get_best_split)

Best feature to split on: 2
 All tests passed.


正如我们在上面看到的，函数返回在根节点处分割的最佳特征是特征 2（"单独生长"）

<a name="5"></a>
## 5 - 构建树

在本节中，我们使用您上面实现的函数，通过连续选择最佳分割特征来生成决策树，直到达到停止条件（最大深度为 2）。

您不需要为此部分实现任何内容。

In [38]:
# Not graded
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    tree.append((current_depth, branch_name, best_feature, node_indices))
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)


In [39]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
